# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
output_data_file="../output_data/cities.csv"
cities_df=pd.read_csv(output_data_file)
cities_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed
0,Constitucion,5,MX,1578141387,85,23.99,-104.67,35.01,4.70
1,Punta Arenas,40,CL,1578141387,66,-53.16,-70.91,51.80,13.87
2,Phuket,20,TH,1578141384,69,7.89,98.40,82.40,5.82
3,Teya,75,MX,1578141387,94,21.05,-89.07,73.99,3.36
4,Rikitea,100,PF,1578141387,82,-23.12,-134.97,77.65,15.48
...,...,...,...,...,...,...,...,...,...
560,Anshun,98,CN,1578141429,93,26.25,105.93,54.93,9.24
561,Coquimbo,90,CL,1578141429,82,-29.95,-71.34,60.80,1.12
562,Stromness,90,CA,1578141429,96,42.87,-79.55,35.01,8.05
563,Pitsunda,53,GE,1578141429,50,43.16,40.34,51.80,2.24


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)


In [4]:

locations=cities_df[['Lat','Lng']]
humidity=cities_df['Humidity'].astype(float)
fig=gmaps.figure()
heatmap_layer=gmaps.heatmap_layer(locations,weights=humidity,dissipating=False,max_intensity=5,point_radius=1)
fig.add_layer(heatmap_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
new_cities_df=pd.DataFrame(cities_df,columns=["City","Max_Temp","Wind_Speed","Cloudiness"])
max_temp=(new_cities_df["Max_Temp"]<=80)&(new_cities_df["Max_Temp"]>70)
wind_speed=new_cities_df["Wind_Speed"]<10
cloudiness=new_cities_df["Cloudiness"]==0
new_cities_df[max_temp&wind_speed&cloudiness]

,City,Max_Temp,Wind_Speed,Cloudiness
85,Barmer,73.17,1.03,0
146,Ibra,79.57,7.85,0
171,Rocha,72.07,7.05,0
225,Chuy,72.59,5.08,0
278,Banda Aceh,76.44,7.58,0
300,Ratnagiri,74.57,5.99,0
383,Santa Vitoria do Palmar,72.59,5.08,0
403,Concordia,72.00,6.93,0
445,Port Macquarie,77.00,6.93,0
448,San Quintin,71.24,6.69,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df=pd.DataFrame(cities_df,columns=["City","Country","Lat","Lng"])
hotel_df["Hotel_Name"]=""
hotel_df


,City,Country,Lat,Lng,Hotel_Name
0,Constitucion,MX,23.99,-104.67,
1,Punta Arenas,CL,-53.16,-70.91,
2,Phuket,TH,7.89,98.40,
3,Teya,MX,21.05,-89.07,
4,Rikitea,PF,-23.12,-134.97,
...,...,...,...,...,...
560,Anshun,CN,26.25,105.93,
561,Coquimbo,CL,-29.95,-71.34,
562,Stromness,CA,42.87,-79.55,
563,Pitsunda,GE,43.16,40.34,


In [7]:

hotels = []
x=0

for cities in hotel_df['City']:
    target_coordinates = str(hotel_df['Lat'][x])+' , '+str(hotel_df['Lng'][x])
    x+=1
    target_radius = 5000
    target_type = 'lodging'
    params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    try:
        response = requests.get(base_url, params=params)
        hotel_results = response.json()
        hotels.append(hotel_results['results'][0]['name'])
        
    except IndexError:
        hotels.append(np.nan)


hotel_df['Hotel Name'] = hotels
hotel_df



,City,Country,Lat,Lng,Hotel_Name,Hotel Name
0,Constitucion,MX,23.99,-104.67,,Hotel Posada San Agustin
1,Punta Arenas,CL,-53.16,-70.91,,Hotel Dreams Del Estrecho
2,Phuket,TH,7.89,98.40,,Rome Place Hotel
3,Teya,MX,21.05,-89.07,,Hacienda Grupo Mérida Sur
4,Rikitea,PF,-23.12,-134.97,,Pension Maro'i
...,...,...,...,...,...,...
560,Anshun,CN,26.25,105.93,,安顺麦吉风格酒店
561,Coquimbo,CL,-29.95,-71.34,,Hotel San Juan
562,Stromness,CA,42.87,-79.55,,Lighthouse RV Resort Inc
563,Pitsunda,GE,43.16,40.34,,Hotel FREEDOM


In [8]:
hotel_df=hotel_df.dropna()
hotel_df

,City,Country,Lat,Lng,Hotel_Name,Hotel Name
0,Constitucion,MX,23.99,-104.67,,Hotel Posada San Agustin
1,Punta Arenas,CL,-53.16,-70.91,,Hotel Dreams Del Estrecho
2,Phuket,TH,7.89,98.40,,Rome Place Hotel
3,Teya,MX,21.05,-89.07,,Hacienda Grupo Mérida Sur
4,Rikitea,PF,-23.12,-134.97,,Pension Maro'i
...,...,...,...,...,...,...
560,Anshun,CN,26.25,105.93,,安顺麦吉风格酒店
561,Coquimbo,CL,-29.95,-71.34,,Hotel San Juan
562,Stromness,CA,42.87,-79.55,,Lighthouse RV Resort Inc
563,Pitsunda,GE,43.16,40.34,,Hotel FREEDOM


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
fig=gmaps.figure()
markers=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))